## 各店舗でよく使われる「2単語の組み合わせ（Bi-gram）」の頻出度

In [ ]:
import pandas as pd
from janome.tokenizer import Tokenizer
from collections import Counter
import time

def load_data(filename='csv/odaiba_reviews.csv'):

    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        print(f"エラー: '{filename}'が見つかりません。")
        print("このスクリプトと同じフォルダにCSVファイルを置いてください。")
        return None

def get_meaningful_tokens(text, tokenizer, stop_words):
    """テキストを形態素解析し、分析に使う単語（名詞, 動詞, 形容詞）の原型リストを返す"""
    tokens = []
    for token in tokenizer.tokenize(text):
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech in ['名詞', '動詞', '形容詞'] and token.surface not in stop_words:
            tokens.append(token.base_form)
    return tokens

# メイン
if __name__ == '__main__':
    df = load_data()

    if df is not None:
        # JanomeのTokenizerとストップワードを準備
        t = Tokenizer()
        stop_words = ['こと', 'もの', 'それ', 'これ', 'さん', 'よう', 'ため',
                       'こちら', 'ところ', '感じ', '利用', '訪問', '注文', '提供', 
                       '料理', 'お店', '感じ', '雰囲気', '時間', '今回', '非常', '思う',
                         'いる', 'する', 'なる', 'ある', 'ない', 'くる', 'いく', '良い', '思う','...']

        # 店舗名でグループ分け
        grouped = df.groupby('shop_name')

        print("--- 店舗ごとの頻出Bi-gram（2単語の組み合わせ）TOP 7 ---\n")
        
        # 各店舗ごとに処理
        for shop_name, group in grouped:
            print(f"★★★ {shop_name} ★★★")

            # その店舗の全口コミを一つに連結
            all_reviews_text = ' '.join(group['review_text'].dropna())

            # 形態素解析を実行し、単語リストを取得
            tokens = get_meaningful_tokens(all_reviews_text, t, stop_words)
            
            # Bi-gram作成処理 
            bigrams = []
            # 単語リストをループし、隣り合う2つの単語をペアにする
            for i in range(len(tokens) - 1):
                bigram = (tokens[i], tokens[i+1]) # タプルとしてペアを作成
                bigrams.append(bigram)
            # Bi-gram作成完了 

            if bigrams:
                # 作成したBi-gramの出現回数をカウント
                bigram_counts = Counter(bigrams)
                
                # 最も多く出現した7つのBi-gramを表示
                for bigram, count in bigram_counts.most_common(7):
                    print(f"  - {bigram[0]}, {bigram[1]}: {count}回")
            else:
                print("  - 分析可能なBi-gramが見つかりませんでした。")

            print("-" * 30)
            time.sleep(0.5) # 1店舗表示するごとに少し待機

--- 店舗ごとの頻出Bi-gram（2単語の組み合わせ）TOP 7 ---

★★★ 1129 by Ogawa ★★★
  - レインボー, ブリッジ: 64回
  - デックス, 東京: 48回
  - 東京, ビーチ: 38回
  - 予約, する: 34回
  - 6, 階: 30回
  - 台場, 海浜: 28回
  - 海浜, 公園: 28回
------------------------------
★★★ 82 築地店 ★★★
  - 築地, 店: 9回
  - 築地, 駅: 8回
  - 82, 築地: 5回
  - 東京, メトロ: 4回
  - ８, ２: 4回
  - ２, 築地: 4回
  - 東, 銀座: 4回
------------------------------
★★★ ABURI 百貫 有明ガーデン店 ★★★
  - 回転, 寿司: 69回
  - 有明, ガーデン: 49回
  - 寿司, 屋: 20回
  - 百, 貫: 17回
  - ABURI, 百: 15回
  - 炙る, 寿司: 12回
  - する, いる: 12回
------------------------------
★★★ ARIAKE ARENA CAFE ★★★
  - 有明, アリーナ: 11回
  - 参加, する: 3回
  - ライブ, 有明: 2回
  - アリーナ, 行く: 2回
  - アリーナ, 内: 2回
  - 日, (: 2回
  - せる, いただく: 2回
------------------------------
★★★ ARIAKE ARENA DINING MOON RIVER ★★★
  - 有明, アリーナ: 6回
  - アリーナ, 併設: 2回
  - 30, 分: 2回
  - イベント, 日: 2回
  - 混雑, する: 2回
  - 清潔, する: 1回
  - する, 気持ち: 1回
------------------------------
★★★ AkeruE ★★★
  - 国際, 展示: 1回
  - 展示, 場: 1回
  - 場, 駅: 1回
  - 駅, 出る: 1回
  - 出る, 左手: 1回
  - 左手, パナソニッククリエイティブミュージアム: 1回
  - パナソ

## 各店舗の口コミを2単語の組み合わせに編集

In [7]:
# 01_preprocess_bigrams.py (コメント修正版)

import pandas as pd
from janome.tokenizer import Tokenizer

# CSVファイルを読み込む関数
def load_data(filename):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        print(f"エラー: '{filename}'が見つかりません。")
        return None

# 文章を単語に分解し、重要な単語（名詞など）だけを抜き出す関数
def get_meaningful_tokens(text, tokenizer, stop_words):
    tokens = []
    for token in tokenizer.tokenize(text):
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech in ['名詞', '動詞', '形容詞'] and token.surface not in stop_words:
            tokens.append(token.base_form)
    return tokens

# 単語リストから2単語のペア（Bi-gram）の文字列を作る関数
def text_to_bigram_string(text, tokenizer, stop_words):
    tokens = get_meaningful_tokens(text, tokenizer, stop_words)
    bigrams = [f"{tokens[i]}_{tokens[i+1]}" for i in range(len(tokens) - 1)]
    return ' '.join(bigrams)

#　メイン処理 
if __name__ == '__main__':
    # 1. 読み込むファイルを設定
    input_filename = 'csv/odaiba_reviews.csv'
    df = load_data(input_filename)

    if df is not None:
        # 2. 分析の準備
        t = Tokenizer()
        stop_words = ['こと', 'もの', 'それ', 'これ', 'さん', 'よう', 
                      'ため', 'こちら', 'ところ', '感じ', '利用', '訪問', 
                      '注文', '提供', '料理', 'お店', '感じ', '雰囲気', '時間', 
                      '今回', '非常', '思う', 'いる', 'する', 'なる', 'ある', 'ない',
                         'くる', 'いく', '良い', '思う', '...']

        # 3. 店舗ごとに口コミを一つの文章にまとめる
        shop_docs = df.groupby('shop_name')['review_text'].apply(lambda x: ' '.join(x.dropna())).reset_index()
        
        print("各店舗の口コミをBi-gramの文字列に変換中...\n")
        
        # 4. 各店舗の文章をBi-gram文字列に変換（進捗表示付き）
        processed_corpus = []
        total_shops = len(shop_docs)

        for index, row in shop_docs.iterrows():
            shop_name = row['shop_name']
            review_text = row['review_text']

            # Bi-gram文字列に変換
            bigram_string = text_to_bigram_string(review_text, t, stop_words)
            processed_corpus.append(bigram_string)
            
            # 進捗を表示
            print(f"  [{index + 1}/{total_shops}] {shop_name} の変換完了．")
        
        shop_docs['bigram_corpus'] = processed_corpus
        
        # 5. 結果を新しいCSVファイルに保存
        output_df = shop_docs[['shop_name', 'bigram_corpus']]
        output_df.to_csv('preprocessed_bigrams.csv', index=False, encoding='utf-8-sig')
        
        print("\n前処理完了。「preprocessed_bigrams.csv」を保存しました。")

各店舗の口コミをBi-gramの文字列に変換中...

  [1/563] 1129 by Ogawa の変換完了．
  [2/563] 82 築地店 の変換完了．
  [3/563] ABURI 百貫 有明ガーデン店 の変換完了．
  [4/563] ARIAKE ARENA CAFE の変換完了．
  [5/563] ARIAKE ARENA DINING MOON RIVER の変換完了．
  [6/563] AkeruE の変換完了．
  [7/563] Ariake Miraie の変換完了．
  [8/563] BBQ PLAY GROUND お台場デックス東京ビーチ店 の変換完了．
  [9/563] BISTRO うしすけ の変換完了．
  [10/563] BLUE TABLE Odaiba の変換完了．
  [11/563] Bistro961 の変換完了．
  [12/563] Bomb! Dogs の変換完了．
  [13/563] Brew33 Bar の変換完了．
  [14/563] CAFE TERIA BLUE の変換完了．
  [15/563] Cha pi の変換完了．
  [16/563] Chez コバラヘッタ 有明ガーデン店 の変換完了．
  [17/563] CoCo壱番屋 ワンザ有明店 の変換完了．
  [18/563] Cuore の変換完了．
  [19/563] D-ラウンジ の変換完了．
  [20/563] DAIBA の変換完了．
  [21/563] DOG DEPT お台場東京ビーチ店 の変換完了．
  [22/563] DOLCE VITA の変換完了．
  [23/563] Deli&Cafe Uran の変換完了．
  [24/563] Domino's Pizza  ARIAKE GARDEN の変換完了．
  [25/563] Eat it! 東京ビッグサイト店 の変換完了．
  [26/563] FARMS' KITCHEN MIKIKE TOKYO ODAIBA の変換完了．
  [27/563] GODIVA アクアシティお台場店 の変換完了．
  [28/563] GODIVA ダイバーシティ東京プラザ店 の変換完了．
  [29/563] GOKUGOKU 有明ガーデン店 の変換完了